# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the RasHdf and RasUtils classes to streamline data extraction, processing, and visualization.

In [ ]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


## Importing ras-commander flexibly (from package or local dev copy)

In [ ]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import init_ras_project, RasHdf, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras

print("ras_commander imported successfully")


In [ ]:
# Define the path to the Muncie project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
muncie_path = current_dir / "example_projects" / "Muncie"
bald_eagle_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging
# Check if Muncie.p03.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = muncie_path / "Muncie.p03.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the Muncie project
    ras_examples = RasExamples()
    ras_examples.extract_project(["Muncie", "BaldEagleCrkMulti2D"])

    # Initialize custom Ras objects
    muncie = RasPrj()
    bald_eagle = RasPrj()

    # Initialize the RAS projects using the custom ras objects
    muncie = init_ras_project(muncie_path, "6.6", ras_instance=muncie)
    logging.info(f"Muncie project initialized with folder: {muncie.project_folder}")

    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    logging.info(f"Bald Eagle project initialized with folder: {bald_eagle.project_folder}")
    
    logging.info(f"Muncie object id: {id(muncie)}")
    logging.info(f"Bald Eagle object id: {id(bald_eagle)}")
    
    # Define the plan number to execute
    plan_number = "03"

    # Set plan keys for both projects
    for project in [muncie, bald_eagle]:
        RasPlan.update_plan_value(plan_number, "run_htab", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_unet", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_postProcess", 1, ras_object=project)
        RasPlan.update_plan_value(plan_number, "run_rasmapper", 0, ras_object=project)

    # Execute Plan 03 using RasCmdr for Muncie
    print(f"Executing Plan {plan_number} for the Muncie project...")
    success_muncie = RasCmdr.compute_plan(plan_number, ras_object=muncie)
    if success_muncie:
        print(f"Plan {plan_number} executed successfully for Muncie.\n")
    else:
        print(f"Plan {plan_number} execution failed for Muncie.\n")
    
    # Execute Plan 03 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_bald_eagle = RasCmdr.compute_plan(plan_number, ras_object=bald_eagle)
    if success_bald_eagle:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("Muncie.p03.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    muncie = RasPrj()
    bald_eagle = RasPrj()
    muncie = init_ras_project(muncie_path, "6.6", ras_instance=muncie)
    bald_eagle = init_ras_project(bald_eagle_path, "6.6", ras_instance=bald_eagle)
    plan_number = "03"

In [ ]:
# Retrieve the HDF file path for Plan 03
results_path_muncie = RasPlan.get_results_path(plan_number, ras_object=muncie)
if results_path_muncie:
    print(f"Results for Plan {plan_number} are located at: {results_path_muncie}\n")
else:
    print(f"No results found for Plan {plan_number}.\n")
    
results_path_baldeagle = RasPlan.get_results_path(plan_number, ras_object=bald_eagle)
if results_path_baldeagle:
    print(f"Results for Plan {plan_number} are located at: {results_path_baldeagle}\n")
else:
    print(f"No results found for Plan {plan_number}.\n")    

In [ ]:
# Define the HDF input path as Plan Number
hdf_input = plan_number

# Initialize RasHdf handler
hdf_handler = RasHdf()

# The remainder of the examples only use muncie, so let's set results_path to muncie
results_path = results_path_muncie



In [ ]:
# Example 1: List all HDF paths with properties
print("Example 1: Listing all HDF paths with properties")
hdf_paths_df = RasHdf.get_hdf_paths_with_properties(hdf_input, ras_object=muncie)
display(hdf_paths_df.head())

In [ ]:
# Example 2: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = RasHdf.get_runtime_data(hdf_input, ras_object=muncie)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

In [ ]:
# Example 3: Get 2D Flow Area Names
print("\nExample 3: Listing 2D Flow Area Names")
flow_area_names = RasHdf.get_2d_flow_area_names(hdf_input, ras_object=muncie)
print("2D Flow Area Names:", flow_area_names)

In [ ]:
# Example 4: Extract 2D Flow Area Attributes
print("\nExample 4: Extracting 2D Flow Area Attributes")
flow_area_attributes_df = RasHdf.get_2d_flow_area_attributes(hdf_input, ras_object=muncie)
if flow_area_attributes_df is not None:
    display(flow_area_attributes_df.head())
else:
    print("No 2D Flow Area attributes found.")

In [ ]:
# Example 5: Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = RasHdf.get_cell_info(hdf_input, ras_object=muncie)
if cell_info_df is not None:
    display(cell_info_df.head())
else:
    print("No Cell Info found.")

In [ ]:
# Example 6: Extract Cell Points
print("\nExample 6: Extracting Cell Points")
cell_points_df = RasHdf.get_cell_points(hdf_input, ras_object=muncie)
if cell_points_df is not None:
    display(cell_points_df.head())
else:
    print("No Cell Points found.")

In [ ]:
# Example 7: Extract Polygon Info and Parts
print("\nExample 7: Extracting Polygon Info and Parts")
polygon_info_df, polygon_parts_df = RasHdf.get_polygon_info_and_parts(hdf_input, ras_object=muncie)
print("Polygon Info:")
if polygon_info_df is not None:
    display(polygon_info_df.head())
else:
    print("No Polygon Info found.")
print("\nPolygon Parts:")
if polygon_parts_df is not None:
    display(polygon_parts_df.head())
else:
    print("No Polygon Parts found.")

In [ ]:
# Example 8: Extract Polygon Points
print("\nExample 8: Extracting Polygon Points")
polygon_points_df = RasHdf.get_polygon_points(hdf_input, ras_object=muncie)
if polygon_points_df is not None:
    display(polygon_points_df.head())
else:
    print("No Polygon Points found.")

In [ ]:
# Example 9: Extract Cells Center Coordinates and Manning's n
print("\nExample 9: Extracting Cells Center Coordinates and Manning's n")
cells_center_coord_df, cells_manning_n_df = RasHdf.get_cells_center_data(hdf_input, ras_object=muncie)
print("Cells Center Coordinates:")
if cells_center_coord_df is not None:
    display(cells_center_coord_df.head())
else:
    print("No Cells Center Coordinates found.")
print("\nCells Manning's n:")
if cells_manning_n_df is not None:
    display(cells_manning_n_df.head())
else:
    print("No Cells Manning's n found.")

In [ ]:
# Example 10: Extract Faces Area Elevation Data
print("\nExample 10: Extracting Faces Area Elevation Data")
faces_elev_df = RasHdf.get_faces_area_elevation_data(hdf_input, ras_object=muncie)
if faces_elev_df is not None:
    display(faces_elev_df.head())
else:
    print("No Faces Area Elevation Data found.")

In [ ]:
# Example 11: Extract Compute Messages as String
print("\nExample 11: Extracting Compute Messages")
compute_messages = RasHdf.extract_string_from_hdf(hdf_input, '/Results/Summary/Compute Messages (text)', ras_object=muncie)
print("Compute Messages:")
print(compute_messages)

In [ ]:
# Example 12: Extract Plan Parameters and Volume Accounting
print("\nExample 12: Extracting Plan Parameters and Volume Accounting Data")
import h5py
import numpy as np

# Extract plan parameters
with h5py.File(str(results_path), 'r') as hdf_file:
    plan_parameters = hdf_file['Plan Data/Plan Parameters']
    
    # List group attributes
    print("Plan Parameters Group Attributes:")
    #for attr_name, attr_value in plan_parameters.attrs.items():
        #print(f"{attr_name}: {attr_value}")
    
    # Extract plan parameters as a DataFrame
    plan_parameters_df = pd.DataFrame([(attr_name, attr_value) for attr_name, attr_value in plan_parameters.attrs.items()], columns=['Attribute', 'Value'])

# Construct the group path for volume accounting data
group_to_list = "Results/Unsteady/Summary/Volume Accounting/Volume Accounting 2D"

# Extract volume accounting data as a DataFrame
volume_accounting_df = RasHdf.get_group_attributes_as_df(results_path, group_to_list)

print("\nPlan Parameters DataFrame:")
display(plan_parameters_df)

print("\nVolume Accounting DataFrame:")
display(volume_accounting_df)

In [ ]:
# Example 13: Listing 2D Flow Area Groups
print("\nExample: Listing 2D Flow Area Groups")

# Get the names of all 2D Flow Area groups
flow_area_group_names = RasHdf.get_2d_flow_area_names(hdf_input, ras_object=muncie)

print("2D Flow Area Groups:")
if flow_area_group_names:
    for name in flow_area_group_names:
        print(f"- {name}")
else:
    print("No 2D Flow Area groups found in the HDF file.")

import h5py

# Additional information about the first group (if any)
if flow_area_group_names:
    first_group = flow_area_group_names[0]
    print(f"\nDetailed information for the first group: {first_group}")
    
    # Remember, use results_path because we are accessing the hdf file and hdf_input is the plan number
    with h5py.File(str(results_path), 'r') as hdf_file:
        group = hdf_file[f'Geometry/2D Flow Areas/{first_group}']
        print("Datasets in this group:")
        for name, item in group.items():
            if isinstance(item, h5py.Dataset):
                print(f"- {name}: Shape {item.shape}, Dtype {item.dtype}")

In [ ]:
# Example 14: Extract Faces Indexes
print("\nExample 13: Extracting Faces Indexes")
cell_indexes_df, facepoint_indexes_df = RasHdf.get_faces_indexes(hdf_input, ras_object=muncie)
print("Faces Cell Indexes:")
if cell_indexes_df is not None:
    display(cell_indexes_df.head())
else:
    print("No Faces Cell Indexes found.")
print("\nFaces FacePoint Indexes:")
if facepoint_indexes_df is not None:
    display(facepoint_indexes_df.head())
else:
    print("No Faces FacePoint Indexes found.")

In [ ]:
# Example 15: Extract Faces Elevation Data
print("\nExample 14: Extracting Faces Elevation Data")
low_elev_centroid_df, min_elevation_df = RasHdf.get_faces_elevation_data(hdf_input, ras_object=muncie)
print("Faces Low Elevation Centroid:")
if low_elev_centroid_df is not None:
    display(low_elev_centroid_df.head())
else:
    print("No Faces Low Elevation Centroid found.")
print("\nFaces Minimum Elevation:")
if min_elevation_df is not None:
    display(min_elevation_df.head())
else:
    print("No Faces Minimum Elevation found.")

In [ ]:
# Example 16: Extract Faces Vector Data
print("\nExample 15: Extracting Faces Vector Data")
faces_vector_df = RasHdf.get_faces_vector_data(hdf_input, ras_object=muncie)
if faces_vector_df is not None:
    display(faces_vector_df.head())
else:
    print("No Faces Vector Data found.")

In [ ]:
# Example 17: Extract Faces Perimeter Data
print("\nExample 16: Extracting Faces Perimeter Data")
perimeter_info_df, perimeter_values_df = RasHdf.get_faces_perimeter_data(hdf_input, ras_object=muncie)
print("Faces Perimeter Info:")
if perimeter_info_df is not None:
    display(perimeter_info_df.head())
else:
    print("No Faces Perimeter Info found.")
print("\nFaces Perimeter Values:")
if perimeter_values_df is not None:
    display(perimeter_values_df.head())
else:
    print("No Faces Perimeter Values found.")

In [ ]:
# Example 18: Extract Infiltration Data
print("\nExample 17: Extracting Infiltration Data")
cell_classifications_df, face_classifications_df, initial_deficit_df, maximum_deficit_df, potential_percolation_rate_df = RasHdf.get_infiltration_data(hdf_input, ras_object=muncie)
print("Infiltration - Cell Classifications:")
if cell_classifications_df is not None:
    display(cell_classifications_df.head())
else:
    print("No Infiltration Cell Classifications found.")
print("\nInfiltration - Face Classifications:")
if face_classifications_df is not None:
    display(face_classifications_df.head())
else:
    print("No Infiltration Face Classifications found.")
print("\nInfiltration - Initial Deficit:")
if initial_deficit_df is not None:
    display(initial_deficit_df.head())
else:
    print("No Infiltration Initial Deficit found.")
print("\nInfiltration - Maximum Deficit:")
if maximum_deficit_df is not None:
    display(maximum_deficit_df.head())
else:
    print("No Infiltration Maximum Deficit found.")
print("\nInfiltration - Potential Percolation Rate:")
if potential_percolation_rate_df is not None:
    display(potential_percolation_rate_df.head())
else:
    print("No Infiltration Potential Percolation Rate found.")

In [ ]:
# Example 19: Extract Percent Impervious Data
print("\nExample 18: Extracting Percent Impervious Data")
cell_classifications_df, face_classifications_df, percent_impervious_df = RasHdf.get_percent_impervious_data(hdf_input, ras_object=muncie)
print("Percent Impervious - Cell Classifications:")
if cell_classifications_df is not None:
    display(cell_classifications_df.head())
else:
    print("No Percent Impervious Cell Classifications found.")
print("\nPercent Impervious - Face Classifications:")
if face_classifications_df is not None:
    display(face_classifications_df.head())
else:
    print("No Percent Impervious Face Classifications found.")
print("\nPercent Impervious:")
if percent_impervious_df is not None:
    display(percent_impervious_df.head())
else:
    print("No Percent Impervious data found.")
    
    # Note: Does not exist in the Muncie Plan 3 example used. 

In [ ]:
# Example 20: Extract Perimeter Data
print("\nExample 19: Extracting Perimeter Data")
perimeter_df = RasHdf.get_perimeter_data(hdf_input, ras_object=muncie)
print("Perimeter Data:")
if perimeter_df is not None:
    display(perimeter_df.head())
else:
    print("No Perimeter Data found.")

In [ ]:
# Example 21: Extract Boundary Condition Lines Attributes
print("\nExample 20: Extracting Boundary Condition Lines Attributes")
bc_lines_df = RasHdf.get_group_attributes_as_df(hdf_input, '/Geometry/Boundary Condition Lines/Attributes', ras_object=muncie)
if bc_lines_df is not None:
    display(bc_lines_df.head())
else:
    print("No Boundary Condition Lines Attributes found.")
    
# None in Muncie plan 3, no output expected.

In [ ]:
# Example 22: Extract Boundary Condition Time Series Data
print("\nExample 22: Extracting Boundary Condition Time Series Data")
bc_time_series_df = RasHdf.get_group_attributes_as_df(hdf_input, '/Geometry/Boundary Condition Lines/Attributes', ras_object=muncie)
if bc_time_series_df is not None:
    display(bc_time_series_df.head())
else:
    print("No Boundary Condition Time Series Data found.")

print("\nAll RasHdf functions have been executed on Plan 03 HDF file.")


# NOTE: Muncie does not have boundary conditions in the HDF file - see how errors are handled. 

In [ ]:
# Example 23: Retrieving 2D Flow Area Solution Times
print("Example 23: Retrieving 2D Flow Area Solution Times")

solution_times = RasHdf.get_2d_area_solution_times(hdf_input, ras_object=muncie)

if solution_times is not None:
    print(f"Retrieved {len(solution_times)} solution times:")
    print(solution_times)
else:
    print("No solution times found.")


In [ ]:
# Example 24: Retrieving 2D Flow Area Solution Time Dates
print("\nExample 24: Retrieving 2D Flow Area Solution Time Dates")

solution_time_dates = RasHdf.get_2d_area_solution_time_dates(hdf_input, ras_object=muncie)

if solution_time_dates is not None:
    print(f"Retrieved {len(solution_time_dates)} solution time dates:")
    print(solution_time_dates)
else:
    print("No solution time dates found.")


In [ ]:
# Example 25: Loading 2D Flow Area Solutions
print("\nExample 25: Loading 2D Flow Area Solutions")

# Load 2D area solutions using the revised method
solutions = RasHdf.load_2d_area_solutions(hdf_input, ras_object=muncie)

# Check if solutions were successfully loaded
if solutions:
    # Access and display solution times
    solution_times_df = solutions.get('solution_times')
    if solution_times_df is not None:
        print("Solution Times:")
        print(solution_times_df.head())
    else:
        print("Solution times not found.")
    
    # Iterate through each 2D Flow Area and display sample data
    for key in solutions:
        if key != 'solution_times':
            print(f"\nData for {key}:")
            area_df = solutions[key]
            print(area_df.head(10))  # Display first 10 records
else:
    print("No solutions loaded.")


In [ ]:
# Example 29: Building Face FacePoints
print("\nExample 29: Building Face FacePoints")

face_facepoints_list = RasHdf.build_face_facepoints(hdf_input, ras_object=muncie)

if face_facepoints_list:
    # Displaying face points indexes for the first face of the first 2D Flow Area
    print("FacePoints Indexes for the first face of the first 2D Flow Area:")
    print(face_facepoints_list[0][0])  # Assuming at least one area and one face
else:
    print("No face facepoints built.")
